<details>
<summary> Anotações </summary>

<li> Caminho para o OCR: "C:\Tesseract OCR\tesseract.exe"

<li> https://github.com/madmaze/pytesseract

<li> https://github.com/tesseract-ocr/tesseract

<li> A detecção de estilo de texto ainda precisa ser desenvolvida mais

<br>

print(pytesseract.image_to_string(Image.open('test.png'), lang='por'))

pdf = pytesseract.image_to_pdf_or_hocr('test.png', extension='pdf')

with open('test.pdf', 'w+b') as f:
    f.write(pdf) # pdf type is bytes by default

</details>

In [41]:
# imports

from pytesseract import pytesseract as pytess
from PIL import Image

import os
from pdf2image import convert_from_path

import cv2
import numpy as np

# executable

folder_executable = r"C:\Tesseract OCR\tesseract.exe"
pytess.tesseract_cmd = folder_executable

In [42]:
# Função para aplicar o pré-processamento na imagem
def preprocess_image(image):
    # Converter para escala de cinza
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Aplicar limiarização adaptativa para segmentar o texto
    threshold = cv2.adaptiveThreshold(
        gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2
    )

    # Aplicar filtro de suavização para remover ruídos
    smoothed = cv2.medianBlur(threshold, 3)

    return smoothed

In [43]:
# Função para detectar se uma região contém texto em negrito
def is_bold(text):
    # Verificar se o texto está em caixa alta (um indicativo comum de negrito)
    return text.isupper()


# Função para detectar se uma região contém texto em itálico
def is_italic(text):
    # Implemente sua lógica de detecção de texto em itálico aqui
    # Isso pode envolver a análise de inclinação, estilo de fonte, etc.
    # Este exemplo é simplificado e apenas verifica se a palavra "italic" está presente no texto
    return "italic" in text.lower()


# Função para formatar o texto
def format_text(page_number, text):
    formatted_text = []

    lines = text.strip().split("\n")
    formatted_text.append("Page # {}:".format(str(page_number)))
    for line in lines:
        if line.isupper():
            formatted_text.append("Title: " + line)
        elif line.islower():
            formatted_text.append("Subtitle: " + line)
        else:
            if is_bold(line):
                formatted_text.append("Bold: " + line)
            elif is_italic(line):
                formatted_text.append("Italic: " + line)
            else:
                formatted_text.append(line)

    return formatted_text

In [44]:
# Caminho do arquivo PDF
filePath = r'C:\\COMPLETE\\PATH\\MMJJ.pdf'

# Converter o PDF em imagens
doc = convert_from_path(filePath)

# Nome do arquivo de saída
output_file = "output.txt"

# Abrir o arquivo para escrita
with open(output_file, "w", encoding="utf-8") as file:
    # Iterar por cada página do documento
    for page_number, page_data in enumerate(doc):
        # Pré-processar a imagem da página
        processed_image = preprocess_image(np.array(page_data))

        # Converter a imagem pré-processada para o formato PIL
        pil_image = Image.fromarray(processed_image)

        # Reconhecer o texto usando o PyTesseract
        txt = pytess.image_to_string(pil_image, lang='por', config='-c preserve_interword_spaces=1')

        # Formatar o texto
        formatted_text = format_text(page_number, txt)

        # Escrever o texto formatado no arquivo
        for line in formatted_text:
            file.write(line + "\n")

print("Texto salvo no arquivo:", output_file)

Texto salvo no arquivo: output.txt
